In [8]:
import numpy as np
from numpy.random import default_rng
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from tabpfn import TabPFNClassifier  # assume installed

rng = default_rng(42)

def make_cov(d=10, rho=0.5, sigma2=1.5):
    """Σ = sigma2 * [(1-rho) I + rho * 11^T]"""
    I = np.eye(d)
    J = np.ones((d, d))
    return sigma2 * ((1 - rho) * I + rho * J)

def make_label_shift_data(n_train=4000, n_test=4000, d=12,
                          train_prior=0.50, test_prior=0.30,
                          rho=0.6, sigma2=2.0, target_mahal=0.8,
                          random_orientation=True, seed=42):
    """
    Generate train/test from two d-dim Gaussians with DIFFERENT PRIORS (label shift),
    and control overlap via the Mahalanobis distance between class means.

    Returns:
      (Xtr, ytr), (Xte, yte), params (dict with mu0, mu1, Sigma, priors, etc.)
    """
    rng_local = default_rng(seed)
    Sigma = make_cov(d, rho=rho, sigma2=sigma2)

    mu0 = np.zeros(d)
    if random_orientation:
        u = rng_local.normal(size=d)
        u = u / np.linalg.norm(u)
    else:
        u = np.ones(d) / np.sqrt(d)

    # Choose mu1 so that (mu1 - mu0)^T Sigma^{-1} (mu1 - mu0) = target_mahal^2
    Sigma_inv = np.linalg.inv(Sigma)
    denom = np.sqrt(u @ Sigma_inv @ u)
    s = (target_mahal / denom) if denom > 0 else 0.0
    mu1 = s * u

    def sample(n, p1):
        y = rng_local.binomial(1, p1, size=n)
        x = np.empty((n, d))
        idx1 = np.where(y == 1)[0]
        idx0 = np.where(y == 0)[0]
        if len(idx1) > 0:
            x[idx1] = rng_local.multivariate_normal(mu1, Sigma, size=len(idx1))
        if len(idx0) > 0:
            x[idx0] = rng_local.multivariate_normal(mu0, Sigma, size=len(idx0))
        return x, y

    Xtr, ytr = sample(n_train, train_prior)
    Xte, yte = sample(n_test, test_prior)

    params_used = dict(
        d=d, rho=rho, sigma2=sigma2, target_mahal=target_mahal,
        train_prior=train_prior, test_prior=test_prior,
        mu0=mu0, mu1=mu1, Sigma=Sigma
    )
    return (Xtr, ytr), (Xte, yte), params_used

def sample_from_gaussians(n, prior1, mu0, mu1, Sigma, seed=None):
    """Sample n points with class-1 prior `prior1` from N(mu0,Sigma)/N(mu1,Sigma)."""
    rng_local = default_rng(seed)
    d = len(mu0)
    y = rng_local.binomial(1, prior1, size=n)
    x = np.empty((n, d))
    idx1 = np.where(y == 1)[0]
    idx0 = np.where(y == 0)[0]
    if len(idx1) > 0:
        x[idx1] = rng_local.multivariate_normal(mu1, Sigma, size=len(idx1))
    if len(idx0) > 0:
        x[idx0] = rng_local.multivariate_normal(mu0, Sigma, size=len(idx0))
    return x, y

def fit_logreg(X_train, y_train, X_test):
    scaler = StandardScaler().fit(X_train)
    Xtr_s = scaler.transform(X_train)
    Xte_s = scaler.transform(X_test)
    clf = LogisticRegression(max_iter=500, solver="lbfgs")
    clf.fit(Xtr_s, y_train)
    p_te = clf.predict_proba(Xte_s)[:, 1]
    return p_te, clf, scaler  # probabilities + model + scaler

def evaluate(y_true, p1):
    yhat = (p1 >= 0.5).astype(int)
    acc = accuracy_score(y_true, yhat)
    auc = roc_auc_score(y_true, p1)
    return acc, auc


In [9]:
from sklearn.model_selection import train_test_split

# ---- Black-Box Shift Estimation (Lipton et al., 2018) ----
def bbse_adjusted_probs(clf, scaler, X_train, y_train, X_test, n_calib=0.2, n_classes=2):
    """
    Return BBSE-adjusted probabilities on X_test.
    We *do not* return/print the estimated prior; only adjusted probs for accuracy/AUC.
    """
    # 1) Build confusion matrix C on a calibration split from TRAIN
    X_tr_cal, _, y_tr_cal, _ = train_test_split(
        X_train, y_train, test_size=(1 - n_calib), stratify=y_train, random_state=123
    )
    X_cal_s = scaler.transform(X_tr_cal)
    yhat_cal = clf.predict(X_cal_s)

    C = np.zeros((n_classes, n_classes))  # rows: predicted, cols: true
    for t in range(n_classes):
        idx = (y_tr_cal == t)
        if idx.sum() > 0:
            counts = np.bincount(yhat_cal[idx], minlength=n_classes)
            C[:, t] = counts / idx.sum()

    # 2) Predicted-label distribution on unlabeled TEST
    Xte_s = scaler.transform(X_test)
    yhat_te = clf.predict(Xte_s)
    q_hat = np.bincount(yhat_te, minlength=n_classes) / len(yhat_te)

    # 3) Estimate test priors internally (no printing), then correct posteriors
    pi_t_hat = np.linalg.pinv(C) @ q_hat
    pi_t_hat = np.clip(pi_t_hat, 1e-6, 1.0)
    pi_t_hat = pi_t_hat / pi_t_hat.sum()

    pi_s = np.bincount(y_train, minlength=n_classes) / len(y_train)

    # Source posteriors from LR
    p_src = clf.predict_proba(Xte_s)  # shape (n, 2)
    ratios = (pi_t_hat / np.maximum(pi_s, 1e-12))[None, :]
    p_adj = p_src * ratios
    p_adj = p_adj / np.maximum(p_adj.sum(axis=1, keepdims=True), 1e-12)
    return p_adj[:, 1]  # adjusted P(Y=1|x) for test points

# ---------------------- Configure a HARD scenario ----------------------
(Xtr, ytr), (Xte, yte), pars = make_label_shift_data(
    n_train=4000, n_test=4000,
    d=12,
    train_prior=0.50, test_prior=0.30,
    rho=0.6,
    sigma2=2.0,
    target_mahal=0.8,
    random_orientation=True,
    seed=7
)

print("Data params:",
      {k: v for k, v in pars.items() if k not in ("mu0", "mu1", "Sigma")})

# ---- Baseline Logistic Regression (trained on TRAIN distribution) ----
p_lr, lr_clf, lr_scaler = fit_logreg(Xtr, ytr, Xte)
acc_lr, auc_lr = evaluate(yte, p_lr)
print(f"LogReg (no correction):        ACC={acc_lr:.3f}, AUC={auc_lr:.3f}")

# ---- Logistic + BBSE (accuracy/AUC only; no prior output) ----
p_lr_bbse = bbse_adjusted_probs(lr_clf, lr_scaler, Xtr, ytr, Xte, n_calib=0.2)
acc_lr_bbse, auc_lr_bbse = evaluate(yte, p_lr_bbse)
print(f"LogReg + BBSE (corrected):     ACC={acc_lr_bbse:.3f}, AUC={auc_lr_bbse:.3f}")

# ---- Oracle Logistic Regression (upper bound for LR) ----
X_oracle, y_oracle = sample_from_gaussians(
    n=len(ytr),
    prior1=pars["test_prior"],
    mu0=pars["mu0"], mu1=pars["mu1"], Sigma=pars["Sigma"],
    seed=999
)
p_oracle, _, _ = fit_logreg(X_oracle, y_oracle, Xte)
acc_oracle, auc_oracle = evaluate(yte, p_oracle)
print(f"Oracle LogReg (upper bound):   ACC={acc_oracle:.3f}, AUC={auc_oracle:.3f}")

# ---- TabPFN (trained on TRAIN distribution, CUDA) ----
scaler_t = StandardScaler().fit(Xtr)
Xtr_s = scaler_t.transform(Xtr).astype("float32")
Xte_s = scaler_t.transform(Xte).astype("float32")

tab = TabPFNClassifier(device="cuda")
tab.fit(Xtr_s, ytr.astype(int))
p_tab = tab.predict_proba(Xte_s)[:, 1]
acc_tab, auc_tab = evaluate(yte, p_tab)
print(f"TabPFN (plain, CUDA):          ACC={acc_tab:.3f}, AUC={auc_tab:.3f}")


Data params: {'d': 12, 'rho': 0.6, 'sigma2': 2.0, 'target_mahal': 0.8, 'train_prior': 0.5, 'test_prior': 0.3}
LogReg (no correction):        ACC=0.645, AUC=0.699
LogReg + BBSE (corrected):     ACC=0.708, AUC=0.699
Oracle LogReg (upper bound):   ACC=0.723, AUC=0.700
TabPFN (plain, CUDA):          ACC=0.641, AUC=0.698
